<a href="https://colab.research.google.com/github/adityarai008/NLP_project/blob/main/B_Tech_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install nltk indic-nlp-library


In [ ]:
!pip install transformers torch

In [ ]:
!pip install sentencepiece

In [ ]:
import os
import glob
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
from html.parser import HTMLParser
nltk.download('stopwords')
nltk.download('punkt')
import html
import torch
nltk.download('indian')
nltk.data.path.append('/root/nltk_data/corpora/stopwords/hindi')

In [ ]:
import nltk
from indicnlp.tokenize import indic_tokenize
nltk.download('words')


In [ ]:
english_vocab = set(nltk.corpus.words.words())


In [ ]:
def clean_text(text):
    text = html.unescape(text)
    text = re.sub(r'http\S+|@\S+|#\S+', '', text)
    text = "".join([i.lower() for i in text if i not in string.punctuation])
    word_tokens = word_tokenize(text)
    stopwords_eng = stopwords.words('english')
    filtered_tokens = [word for word in word_tokens if word not in stopwords_eng]
    cleaned_text = " ".join(filtered_tokens)
    return cleaned_text


In [ ]:
files = glob.glob("./drive/MyDrive/Hindi-english/*.txt")
cleaned_texts = []
for file in files:
        with open(file, "r", encoding="utf-8") as input_file:
            text = input_file.read()
        
        cleaned_text = clean_text(text)

        cleaned_texts.append(cleaned_text)
    


In [ ]:
print(cleaned_texts)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("salesken/translation-hi-en")
model = AutoModelForSeq2SeqLM.from_pretrained("salesken/translation-hi-en")

In [ ]:
def translate_text(text):
    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text


In [ ]:

translated_texts = []
for text in cleaned_texts:
    translated_text = translate_text(text)
    
    print(translated_text)
    translated_texts.append(translated_text)

    

In [ ]:
print(translated_texts)

In [ ]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:

nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text, entities

texts = []
entities_list = []
for text in translated_texts:
    processed_text, entities = preprocess_text(text)
    texts.append(processed_text)
    entities_list.extend(entities)
print(texts)
print(entities_list)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

k = 5  # Choose the appropriate number of clusters for your data
kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=5, verbose=1)
kmeans.fit(X)
labels = kmeans.labels_
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(texts[i])
print(clusters)


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


In [ ]:

def summarize(text, entities):
    # Include important entities in the summarization input
    text_with_entities = f"{', '.join(entities)} - {text}"
    inputs = tokenizer.encode("summarize: " + text_with_entities, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0])
    return summary

cluster_summaries = {}
# Generate summaries for each cluster
cluster_summaries = {}
for cluster_id, cluster_texts in clusters.items():
    combined_text = ""
    combined_entities = []
    for text in cluster_texts:
        processed_text, entities = preprocess_text(text)
        combined_text += processed_text + " "
        combined_entities.extend(entities)
    summary = summarize(combined_text, combined_entities)
    cluster_summaries[cluster_id] = summary

In [ ]:
combined_text = ""
for cluster_id, summary in cluster_summaries.items():
    print(f"Cluster {cluster_id} summary:")
    combined_text += (summary)
    print(summary)
    print("\n")
